In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics


In [2]:
df=pd.read_csv('Weather_Data.csv')
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes


In [3]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [4]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)



In [5]:
df_sydney_processed.drop('Date',axis=1,inplace=True)
df_sydney_processed = df_sydney_processed.astype(float)
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']


In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=42)


In [7]:
LinearReg= LinearRegression()
LinearReg.fit(X_train, y_train)

LinearRegression()

In [8]:
predictions_lr= LinearReg.predict(X_test)

In [9]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


LinearRegression_MAE = mean_absolute_error(y_test, predictions_lr)
LinearRegression_MSE =  mean_squared_error(y_test, predictions_lr)
LinearRegression_R2 = r2_score(y_test, predictions_lr)
metric_df = pd.DataFrame({
    'Metric': ['Mean Absolute Error (MAE)', 'Mean Squared Error (MSE)', 'R-squared (R2)'],
    'Value': [LinearRegression_MAE, LinearRegression_MSE, LinearRegression_R2]
})
metric_df

,Metric,Value
0,Mean Absolute Error (MAE),0.270497
1,Mean Squared Error (MSE),0.131713
2,R-squared (R2),0.336791


In [10]:
knn_classifier = KNeighborsClassifier(n_neighbors=4)

# Train the model
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4)

In [11]:
pred= knn_classifier.predict(X_test)
KNN_Accuracy_Score = accuracy_score(y_test, pred)


KNN_JaccardIndex = jaccard_score(y_test, pred, average='binary')  # For binary classification
KNN_F1_Score = f1_score(y_test, pred)

In [12]:
Tree =  DecisionTreeClassifier()
Tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [13]:
preds= Tree.predict(X_test)

DecisionTree_Accuracy_Score = accuracy_score(y_test, preds)


DecisionTree_JaccardIndex = jaccard_score(y_test, preds, average='binary')  # For binary classification
DecisionTree_F1_Score = f1_score(y_test, preds)

In [14]:
LR = LogisticRegression(solver='liblinear')

# Train the Logistic Regression model
LR.fit(X_train, y_train)
predictions_LR = LR.predict(X_test)
predict_proba = LR.predict_proba(X_test)


In [15]:

logistics_accuracy = accuracy_score(y_test, predictions_LR)
logistics_jaccard_score= jaccard_score(y_test, predictions_LR, average='binary')
logistics_F1score=f1_score(y_test, predictions_LR)
logloss = log_loss(y_test, predict_proba)


In [16]:
from sklearn.svm import SVC
SVM = SVC()
SVM.fit(X_train, y_train)
predictions_svm = SVM.predict(X_test)




In [17]:
SVM_accuracy= accuracy_score(y_test, predictions_svm)
SVM_jaccard= jaccard_score(y_test, predictions_svm, average='binary')
SVM_F1= f1_score(y_test, predictions_svm)

In [18]:
metrics_data = {
    'Model': ['KNN','Decision Tree','Logistic Regression', 'SVM'],

    'Accuracy': [accuracy_score(y_test, pred),
                 accuracy_score(y_test, preds),
                 accuracy_score(y_test, predictions_LR),
                 accuracy_score(y_test, predictions_svm)],
    'Jaccard Index': [jaccard_score(y_test, pred),
                      jaccard_score(y_test, preds),
                      jaccard_score(y_test, predictions_LR),
                      jaccard_score(y_test, predictions_svm)],
    'F1-Score': [f1_score(y_test, pred),
                 f1_score(y_test, preds),
                 f1_score(y_test, predictions_LR),
                 f1_score(y_test, predictions_svm)],
    'LogLoss': [np.nan,
               np.nan,
            log_loss(y_test, predict_proba),
               np.nan]

}

metrics_df = pd.DataFrame(metrics_data)
metrics_df.set_index('Model', inplace=True)
metrics_df


,Accuracy,Jaccard Index,F1-Score,LogLoss
Model,,,,
KNN,0.792366,0.349282,0.517730,NaN
Decision Tree,0.778626,0.433594,0.604905,NaN
Logistic Regression,0.830534,0.473934,0.643087,0.411457
SVM,0.726718,0.000000,0.000000,NaN
